<a href="https://colab.research.google.com/github/drshahizan/Python_EDA/blob/main/assignment/ass4/bdm/RAM/feature_eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 4: Feature Engineering
##Dataset -- NIFTY-50 Stock Market Data (2000 - 2021)

> Group Members:

 | Name          | Matric Number  |
 | ------------- | -------------- |
 | MOHAMMED RAZA ASFAK CHIDIMAR     | MCS231004       |
 | AYAZ RAHMAN BHUIYAN    | MCS232001        |
 | MUSAB IBNE AHMAD  | MCS231017        |
 | HUSSEIN YUSUF SHEIKH MOHAMED   | MCS231024       |


> The data is the price history and trading volumes of the fifty stocks in the index NIFTY 50 from NSE (National Stock Exchange) India. All datasets are at a day-level with pricing and trading values split across .cvs files for each stock along with a metadata file with some macro-information about the stocks itself. The data spans from 1st January, 2000 to 30th April, 2021.


>   We will delve into the dataset's organization, discern any discernible patterns, trends, or anomalies, and closely examine the distributions and relationships of variables to uncover potential insights.





##Loading the Dataset

###Import Data from Kaggle using Kaggle API

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
! pip install kaggle
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [5]:
!cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d rohanrao/nifty50-stock-market-data

 49% 9.00M/18.4M [00:00<00:00, 80.6MB/s]
100% 18.4M/18.4M [00:00<00:00, 120MB/s] 


In [6]:
!unzip nifty50-stock-market-data.zip

Archive:  nifty50-stock-market-data.zip
  inflating: ADANIPORTS.csv          
  inflating: ASIANPAINT.csv          
  inflating: AXISBANK.csv            
  inflating: BAJAJ-AUTO.csv          
  inflating: BAJAJFINSV.csv          
  inflating: BAJFINANCE.csv          
  inflating: BHARTIARTL.csv          
  inflating: BPCL.csv                
  inflating: BRITANNIA.csv           
  inflating: CIPLA.csv               
  inflating: COALINDIA.csv           
  inflating: DRREDDY.csv             
  inflating: EICHERMOT.csv           
  inflating: GAIL.csv                
  inflating: GRASIM.csv              
  inflating: HCLTECH.csv             
  inflating: HDFC.csv                
  inflating: HDFCBANK.csv            
  inflating: HEROMOTOCO.csv          
  inflating: HINDALCO.csv            
  inflating: HINDUNILVR.csv          
  inflating: ICICIBANK.csv           
  inflating: INDUSINDBK.csv          
  inflating: INFRATEL.csv            
  inflating: INFY.csv                
  inflatin

###Importing Libraries

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline
sns.set()
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from geopy.distance import geodesic

###Data Importing

In [8]:
df = pd.read_csv("/content/NIFTY50_all.csv")

In [9]:
df

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
0,2007-11-27,MUNDRAPORT,EQ,440.00,770.00,1050.00,770.00,959.00,962.90,984.72,27294366,2.687719e+15,NaN,9859619.0,0.3612
1,2007-11-28,MUNDRAPORT,EQ,962.90,984.00,990.00,874.00,885.00,893.90,941.38,4581338,4.312765e+14,NaN,1453278.0,0.3172
2,2007-11-29,MUNDRAPORT,EQ,893.90,909.00,914.75,841.00,887.00,884.20,888.09,5124121,4.550658e+14,NaN,1069678.0,0.2088
3,2007-11-30,MUNDRAPORT,EQ,884.20,890.00,958.00,890.00,929.00,921.55,929.17,4609762,4.283257e+14,NaN,1260913.0,0.2735
4,2007-12-03,MUNDRAPORT,EQ,921.55,939.75,995.00,922.00,980.00,969.30,965.65,2977470,2.875200e+14,NaN,816123.0,0.2741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235187,2021-04-26,ZEEL,EQ,188.00,190.60,191.10,185.10,186.70,186.40,187.35,8542755,1.600451e+14,52374.0,2340188.0,0.2739
235188,2021-04-27,ZEEL,EQ,186.40,188.00,192.95,186.80,188.80,188.15,189.41,14247767,2.698636e+14,73673.0,5425957.0,0.3808
235189,2021-04-28,ZEEL,EQ,188.15,188.80,190.60,187.10,188.95,189.10,188.85,8429439,1.591917e+14,44056.0,2413974.0,0.2864
235190,2021-04-29,ZEEL,EQ,189.10,190.80,191.65,186.00,186.60,186.55,187.44,9483009,1.777471e+14,60932.0,2744472.0,0.2894


##Exploratory Data Analysis (EDA)

###Dataset overview

In [10]:
df.columns

Index(['Date', 'Symbol', 'Series', 'Prev Close', 'Open', 'High', 'Low', 'Last',
       'Close', 'VWAP', 'Volume', 'Turnover', 'Trades', 'Deliverable Volume',
       '%Deliverble'],
      dtype='object')

In [11]:
df.shape

(235192, 15)

In [12]:
df.head(5)

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
0,2007-11-27,MUNDRAPORT,EQ,440.00,770.00,1050.00,770.0,959.0,962.90,984.72,27294366,2.687719e+15,NaN,9859619.0,0.3612
1,2007-11-28,MUNDRAPORT,EQ,962.90,984.00,990.00,874.0,885.0,893.90,941.38,4581338,4.312765e+14,NaN,1453278.0,0.3172
2,2007-11-29,MUNDRAPORT,EQ,893.90,909.00,914.75,841.0,887.0,884.20,888.09,5124121,4.550658e+14,NaN,1069678.0,0.2088
3,2007-11-30,MUNDRAPORT,EQ,884.20,890.00,958.00,890.0,929.0,921.55,929.17,4609762,4.283257e+14,NaN,1260913.0,0.2735
4,2007-12-03,MUNDRAPORT,EQ,921.55,939.75,995.00,922.0,980.0,969.30,965.65,2977470,2.875200e+14,NaN,816123.0,0.2741


In [13]:
df.tail(3)

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
235189,2021-04-28,ZEEL,EQ,188.15,188.8,190.60,187.10,188.95,189.10,188.85,8429439,1.591917e+14,44056.0,2413974.0,0.2864
235190,2021-04-29,ZEEL,EQ,189.10,190.8,191.65,186.00,186.60,186.55,187.44,9483009,1.777471e+14,60932.0,2744472.0,0.2894
235191,2021-04-30,ZEEL,EQ,186.55,185.3,190.95,183.65,185.00,185.60,187.53,11435285,2.144440e+14,62607.0,3323909.0,0.2907


In [14]:
df.describe()

,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
count,235192.000000,235192.000000,235192.000000,235192.000000,235192.000000,235192.000000,235192.00000,2.351920e+05,2.351920e+05,1.203440e+05,2.191150e+05,219115.000000
mean,1266.196349,1267.759708,1286.581440,1247.488465,1266.388302,1266.554351,1267.13230,3.045903e+06,1.610138e+14,6.196427e+04,1.315098e+06,0.502997
std,2581.370320,2585.259609,2619.649216,2546.621396,2581.392543,2582.140942,2582.69998,7.333981e+06,3.298085e+14,6.866457e+04,2.831670e+06,0.190019
min,0.000000,8.500000,9.750000,8.500000,9.100000,9.150000,9.21000,3.000000e+00,1.047000e+07,1.100000e+01,5.000000e+00,0.023600
25%,274.300000,275.000000,279.500000,269.600000,274.400000,274.350000,274.69750,2.190095e+05,1.612816e+13,2.183400e+04,1.253830e+05,0.364700
50%,566.500000,567.025000,576.900000,556.500000,567.000000,566.700000,566.94000,1.010938e+06,6.832603e+13,4.406800e+04,5.017560e+05,0.511000
75%,1242.200000,1243.312500,1263.000000,1221.650000,1242.900000,1242.400000,1242.66250,3.019851e+06,1.863835e+14,7.893550e+04,1.452233e+06,0.638400
max,32861.950000,33399.950000,33480.000000,32468.100000,32849.000000,32861.950000,32975.24000,4.810589e+08,3.564334e+16,1.643015e+06,2.325307e+08,1.000000


In [15]:
df.describe(include='object')

,Date,Symbol,Series
count,235192,235192,235192
unique,5306,65,1
top,2018-08-13,IOC,EQ
freq,49,5306,235192


###Checking And Managing Nulls

In [16]:
null_counts = df.isnull().sum()
print("Null counts in each column:")
print(null_counts)

Null counts in each column:
Date                       0
Symbol                     0
Series                     0
Prev Close                 0
Open                       0
High                       0
Low                        0
Last                       0
Close                      0
VWAP                       0
Volume                     0
Turnover                   0
Trades                114848
Deliverable Volume     16077
%Deliverble            16077
dtype: int64


Nulls in Last three column doesn't affect our task aims so we will not replace it or delete the records

We will just remove Trades Column because it has to many null values

In [17]:
df = df.drop('Trades', axis=1)


In [18]:
df.isnull().sum()

Date                      0
Symbol                    0
Series                    0
Prev Close                0
Open                      0
High                      0
Low                       0
Last                      0
Close                     0
VWAP                      0
Volume                    0
Turnover                  0
Deliverable Volume    16077
%Deliverble           16077
dtype: int64

###Data Type

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235192 entries, 0 to 235191
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Date                235192 non-null  object 
 1   Symbol              235192 non-null  object 
 2   Series              235192 non-null  object 
 3   Prev Close          235192 non-null  float64
 4   Open                235192 non-null  float64
 5   High                235192 non-null  float64
 6   Low                 235192 non-null  float64
 7   Last                235192 non-null  float64
 8   Close               235192 non-null  float64
 9   VWAP                235192 non-null  float64
 10  Volume              235192 non-null  int64  
 11  Turnover            235192 non-null  float64
 12  Deliverable Volume  219115 non-null  float64
 13  %Deliverble         219115 non-null  float64
dtypes: float64(10), int64(1), object(3)
memory usage: 25.1+ MB


Data Type are Already in Good structure so will not update any datatype

###Duplicate Value

In [20]:
df.duplicated().sum()


0

In [21]:
df.drop_duplicates()


,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Deliverable Volume,%Deliverble
0,2007-11-27,MUNDRAPORT,EQ,440.00,770.00,1050.00,770.00,959.00,962.90,984.72,27294366,2.687719e+15,9859619.0,0.3612
1,2007-11-28,MUNDRAPORT,EQ,962.90,984.00,990.00,874.00,885.00,893.90,941.38,4581338,4.312765e+14,1453278.0,0.3172
2,2007-11-29,MUNDRAPORT,EQ,893.90,909.00,914.75,841.00,887.00,884.20,888.09,5124121,4.550658e+14,1069678.0,0.2088
3,2007-11-30,MUNDRAPORT,EQ,884.20,890.00,958.00,890.00,929.00,921.55,929.17,4609762,4.283257e+14,1260913.0,0.2735
4,2007-12-03,MUNDRAPORT,EQ,921.55,939.75,995.00,922.00,980.00,969.30,965.65,2977470,2.875200e+14,816123.0,0.2741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235187,2021-04-26,ZEEL,EQ,188.00,190.60,191.10,185.10,186.70,186.40,187.35,8542755,1.600451e+14,2340188.0,0.2739
235188,2021-04-27,ZEEL,EQ,186.40,188.00,192.95,186.80,188.80,188.15,189.41,14247767,2.698636e+14,5425957.0,0.3808
235189,2021-04-28,ZEEL,EQ,188.15,188.80,190.60,187.10,188.95,189.10,188.85,8429439,1.591917e+14,2413974.0,0.2864
235190,2021-04-29,ZEEL,EQ,189.10,190.80,191.65,186.00,186.60,186.55,187.44,9483009,1.777471e+14,2744472.0,0.2894


###Checking Unique Values

In [22]:
for column in df.columns:
    print(f"Unique values in {column}: {df[column].unique()}")


Unique values in Date: ['2007-11-27' '2007-11-28' '2007-11-29' ... '2007-11-22' '2007-11-23'
 '2007-11-26']
Unique values in Symbol: ['MUNDRAPORT' 'ADANIPORTS' 'ASIANPAINT' 'UTIBANK' 'AXISBANK' 'BAJAJ-AUTO'
 'BAJAJFINSV' 'BAJAUTOFIN' 'BAJFINANCE' 'BHARTI' 'BHARTIARTL' 'BPCL'
 'BRITANNIA' 'CIPLA' 'COALINDIA' 'DRREDDY' 'EICHERMOT' 'GAIL' 'GRASIM'
 'HCLTECH' 'HDFC' 'HDFCBANK' 'HEROHONDA' 'HEROMOTOCO' 'HINDALC0'
 'HINDALCO' 'HINDLEVER' 'HINDUNILVR' 'ICICIBANK' 'INDUSINDBK' 'INFOSYSTCH'
 'INFY' 'IOC' 'ITC' 'JSWSTL' 'JSWSTEEL' 'KOTAKMAH' 'KOTAKBANK' 'LT' 'M&M'
 'MARUTI' 'NESTLEIND' 'NTPC' 'ONGC' 'POWERGRID' 'RELIANCE' 'SBIN'
 'SHREECEM' 'SUNPHARMA' 'TELCO' 'TATAMOTORS' 'TISCO' 'TATASTEEL' 'TCS'
 'TECHM' 'TITAN' 'ULTRACEMCO' 'UNIPHOS' 'UPL' 'SESAGOA' 'SSLT' 'VEDL'
 'WIPRO' 'ZEETELE' 'ZEEL']
Unique values in Series: ['EQ']
Unique values in Prev Close: [ 440.    962.9   893.9  ... 1452.85 1454.4  1194.4 ]
Unique values in Open: [770.   984.   909.   ... 435.85 433.65 223.65]
Unique values in Hi

Droping Series column because it has same value so no need to have it

In [23]:
df = df.drop('Series', axis=1)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235192 entries, 0 to 235191
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Date                235192 non-null  object 
 1   Symbol              235192 non-null  object 
 2   Prev Close          235192 non-null  float64
 3   Open                235192 non-null  float64
 4   High                235192 non-null  float64
 5   Low                 235192 non-null  float64
 6   Last                235192 non-null  float64
 7   Close               235192 non-null  float64
 8   VWAP                235192 non-null  float64
 9   Volume              235192 non-null  int64  
 10  Turnover            235192 non-null  float64
 11  Deliverable Volume  219115 non-null  float64
 12  %Deliverble         219115 non-null  float64
dtypes: float64(10), int64(1), object(2)
memory usage: 23.3+ MB


##Feature Selection

###correlation matrix

In [24]:
correlation_matrix = df.corr()
print("Correlation Matrix:")
print(correlation_matrix)

Correlation Matrix:
                    Prev Close      Open      High       Low      Last  \
Prev Close            1.000000  0.999718  0.999616  0.999601  0.999505   
Open                  0.999718  1.000000  0.999857  0.999863  0.999748   
High                  0.999616  0.999857  1.000000  0.999790  0.999881   
Low                   0.999601  0.999863  0.999790  1.000000  0.999868   
Last                  0.999505  0.999748  0.999881  0.999868  1.000000   
Close                 0.999518  0.999761  0.999892  0.999878  0.999991   
VWAP                  0.999622  0.999867  0.999937  0.999929  0.999950   
Volume               -0.118742 -0.118779 -0.118595 -0.118914 -0.118713   
Turnover              0.052888  0.052704  0.053697  0.052144  0.053110   
Deliverable Volume   -0.132987 -0.133025 -0.132979 -0.133141 -0.133051   
%Deliverble          -0.009034 -0.009078 -0.009962 -0.008543 -0.009367   

                       Close      VWAP    Volume  Turnover  \
Prev Close          0.999518 

<ipython-input-24-86e2661ca6d7>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = df.corr()


Identifying highly correlated features

In [25]:
highly_correlated_features = set()
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            colname = correlation_matrix.columns[i]
            highly_correlated_features.add(colname)

print("\nHighly Correlated Features:")
print(highly_correlated_features)


Highly Correlated Features:
{'Close', 'High', 'VWAP', 'Last', 'Open', 'Low', 'Deliverable Volume'}


1. High: The highest price of the stock during a given trading day.
2. Open: The opening price of the stock at the beginning of the trading day.
3. Low: The lowest price of the stock during a given trading day.
4. Last: The last traded price of the stock during a given trading day.
5. Close: The closing price of the stock at the end of the trading day.
6. Deliverable Volume: The volume of shares that were traded and physically delivered (not settled by a futures or options contract).
7. VWAP (Volume Weighted Average Price): A measure of the average price of a stock, weighted by trading volume.

> These features are likely to be highly correlated because they are all based on stock prices and trading volumes. For example, it's common for the opening price to be close to the closing price, and the highest and lowest prices during the day to be related. Similarly, VWAP is influenced by both the price and volume of trades.

> When dealing with highly correlated features, it's often beneficial to remove some of them to avoid multicollinearity, which can affect the stability and interpretability of statistical models. In your case, it seems reasonable to remove some redundant features to simplify the dataset and potentially improve the performance of your analysis or modeling.

###New DataFrame containing only the selected features

In [26]:
selected_features = df.drop(highly_correlated_features, axis=1)

print("\nSelected Features DataFrame:")
print(selected_features.columns)


Selected Features DataFrame:
Index(['Date', 'Symbol', 'Prev Close', 'Volume', 'Turnover', '%Deliverble'], dtype='object')


##Feature Preprocessing

###Another Libraries

In [27]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler


In [45]:
!pip install --upgrade swifter


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16506 sha256=1f03f0848a6a84c6c237009531e313c137668bee8979d1b6c24dc744b188b323
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa3709437065278dc534ec1b8d2ad41b443
Successfully built swifter


In [46]:
import swifter

###Information about missing values

In [28]:
print("Missing Values Before Imputation:")
print(selected_features.isnull().sum())

Missing Values Before Imputation:
Date               0
Symbol             0
Prev Close         0
Volume             0
Turnover           0
%Deliverble    16077
dtype: int64


###Impute missing values

In [29]:
non_numeric_columns = selected_features.select_dtypes(exclude=['number']).columns

numeric_columns = selected_features.columns.difference(non_numeric_columns)
imputer = SimpleImputer(strategy='median')
selected_features_imputed = selected_features.copy()
selected_features_imputed[numeric_columns] = imputer.fit_transform(selected_features[numeric_columns])

print("\nMissing Values After Imputation:")
print(selected_features_imputed.isnull().sum())


Missing Values After Imputation:
Date           0
Symbol         0
Prev Close     0
Volume         0
Turnover       0
%Deliverble    0
dtype: int64


###Normalize the data

In [30]:
scaler = StandardScaler()
selected_features_scaled = pd.DataFrame(scaler.fit_transform(selected_features_imputed[numeric_columns]), columns=numeric_columns)


###Combine the non-numeric columns with the scaled numeric columns

In [31]:
selected_features_scaled = pd.concat([selected_features_imputed[non_numeric_columns], selected_features_scaled], axis=1)
# Summary statistics after preprocessing
print("\nSummary Statistics After Preprocessing:")
print(selected_features_scaled.describe())


Summary Statistics After Preprocessing:
        %Deliverble    Prev Close      Turnover        Volume
count  2.351920e+05  2.351920e+05  2.351920e+05  2.351920e+05
mean  -1.131107e-16  9.667577e-18  1.208447e-17  9.667577e-19
std    1.000002e+00  1.000002e+00  1.000002e+00  1.000002e+00
min   -2.616631e+00 -4.905143e-01 -4.882050e-01 -4.153143e-01
25%   -6.882758e-01 -3.842527e-01 -4.393033e-01 -3.854523e-01
50%    4.065020e-02 -2.710567e-01 -2.810358e-01 -2.774713e-01
75%    6.807094e-01 -9.295993e-03  7.692240e-02 -3.552278e-03
max    2.706655e+00  1.223994e+01  1.075849e+02  6.517798e+01


##Feature Transformation

###Information about data types

In [32]:
print("Data Types Before Transformation:")
print(selected_features_scaled.dtypes)


Data Types Before Transformation:
Date            object
Symbol          object
%Deliverble    float64
Prev Close     float64
Turnover       float64
Volume         float64
dtype: object


###One-hot encode categorical features

In [43]:
categorical_columns = selected_features_scaled.select_dtypes(include=['object']).columns
selected_features_encoded = pd.get_dummies(selected_features_scaled, columns=categorical_columns, drop_first=True)

###Logarithmic transformation for skewed data

In [47]:
skewed_columns = selected_features_encoded.apply(lambda x: x.skew()).sort_values(ascending=False)
skewed_columns = skewed_columns[abs(skewed_columns) > 0.5].index
selected_features_transformed = selected_features_encoded.copy()
selected_features_transformed[skewed_columns] = selected_features_encoded[skewed_columns].swifter.apply(lambda x: np.log1p(x))


###Summary statistics after transformation

In [ ]:
print("\nData Types After Transformation:")
print(selected_features_transformed.dtypes)

print("\nSummary Statistics After Transformation:")
print(selected_features_transformed.describe())


Data Types After Transformation:
%Deliverble        float64
Prev Close         float64
Turnover           float64
Volume             float64
Date_2000-01-04    float16
                    ...   
Symbol_UTIBANK     float16
Symbol_VEDL        float16
Symbol_WIPRO       float16
Symbol_ZEEL        float16
Symbol_ZEETELE     float16
Length: 5373, dtype: object

Summary Statistics After Transformation:


##Feature Creation

###Existing features

In [36]:
print("Existing Features:")
print(selected_features_transformed.head())

Existing Features:
   %Deliverble  Prev Close  Turnover    Volume  Date_2000-01-04  \
0    -0.776052   -0.385753  2.158847  1.460085              0.0   
1    -1.015938   -0.124990  0.598537  0.190091              0.0   
2    -1.606930   -0.155747  0.637416  0.249489              0.0   
3    -1.254188   -0.160148  0.593607  0.193290              0.0   
4    -1.250917   -0.143308  0.324671 -0.009375              0.0   

   Date_2000-01-05  Date_2000-01-06  Date_2000-01-07  Date_2000-01-10  \
0              0.0              0.0              0.0              0.0   
1              0.0              0.0              0.0              0.0   
2              0.0              0.0              0.0              0.0   
3              0.0              0.0              0.0              0.0   
4              0.0              0.0              0.0              0.0   

   Date_2000-01-11  ...  Symbol_TISCO  Symbol_TITAN  Symbol_ULTRACEMCO  \
0              0.0  ...           0.0           0.0              

###Feature creation

In [39]:
window_size = 7
selected_features_transformed['Moving_Average'] = selected_features_transformed['Prev Close'].rolling(window=window_size).mean()

<ipython-input-39-57bdceeeabe0>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  selected_features_transformed['Moving_Average'] = selected_features_transformed['Prev Close'].rolling(window=window_size).mean()


###Statistics after feature creation

In [40]:
print("\nSummary Statistics After Feature Creation:")
print(selected_features_transformed.describe())



Summary Statistics After Feature Creation:
        %Deliverble     Prev Close       Turnover         Volume  \
count  2.351920e+05  235192.000000  235192.000000  235192.000000   
mean  -1.131107e-16      -0.168750      -0.176359      -0.156899   
std    1.000002e+00       0.473787       0.510561       0.463477   
min   -2.616631e+00      -0.674353      -0.669831      -0.536681   
25%   -6.882758e-01      -0.484919      -0.578575      -0.486869   
50%    4.065020e-02      -0.316159      -0.329944      -0.324998   
75%    6.807094e-01      -0.009339       0.074107      -0.003559   
max    2.706655e+00       2.583238       4.687532       4.192348   

       Date_2000-01-04  Date_2000-01-05  Date_2000-01-06  Date_2000-01-07  \
count    235192.000000    235192.000000    235192.000000    235192.000000   
mean          0.000000         0.000000         0.000000         0.000000   
std           0.000000         0.000000         0.000000         0.000000   
min           0.000000         0.00

##Visualization

###Plot histograms for selected features

In [1]:
selected_features_transformed.hist(figsize=(12, 10), bins=20)
plt.suptitle("Histograms of Transformed Features", y=1.02)
plt.show()

NameError: name 'selected_features_transformed' is not defined

###Plot pair plot for selected features

In [ ]:
sns.pairplot(selected_features_transformed)
plt.suptitle("Pair Plot of Transformed Features", y=1.02)
plt.show()

##Conclusion

In conclusion, the analysis and feature engineering steps performed on the given dataset aimed to prepare the data for modeling by addressing various aspects such as missing values, outliers, and non-numeric data. Here is a summary of the findings and feature engineering steps:

**Exploratory Data Analysis (EDA)**:

The dataset includes financial market data for a stock, containing information such as date, symbol, series, previous close, open, high, low, last, close, VWAP, volume, turnover, trades, deliverable volume, and the percentage of deliverable.

**Feature Selection:**

Identified highly correlated features and removed them to prevent multicollinearity.
Dropped the 'Trades' column as part of feature selection.

**Feature Preprocessing:**

Imputed missing values using the median strategy for numeric columns.
Standardized numeric data to bring features to a common scale.
Addressed potential outliers, though the specific method was not provided in the code.

**Feature Transformation:**

Applied one-hot encoding to categorical features using pd.get_dummies.
Utilized the logarithmic transformation for skewed data using np.log1p.

**Visualization:**

Visualized the transformed data using histograms and a pair plot to gain insights into feature distributions and relationships.
Decisions were made based on common practices in data preprocessing and feature engineering, considering the characteristics of financial market data. The goal was to create a clean and transformed dataset that is suitable for modeling. The steps taken aimed to handle missing values, address multicollinearity, and prepare the data for machine learning algorithms.